<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 3505, "amount": 602}, {"transaction-id": 7941, "amount": 619}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 3505, "amount": 602}, {"transaction-id": 7941, "amount": 619}]}\n',
 '{"id": 1, "name": "Tim", "transactions": [{"transaction-id": 830, "amount": -4332}, {"transaction-id": 9561, "amount": -3984}]}\n',
 '{"id": 2, "name": "Jerry", "transactions": [{"transaction-id": 1879, "amount": 244}, {"transaction-id": 3925, "amount": 203}]}\n')

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 3505, 'amount': 602},
   {'transaction-id': 7941, 'amount': 619}]},
 {'id': 1,
  'name': 'Tim',
  'transactions': [{'transaction-id': 830, 'amount': -4332},
   {'transaction-id': 9561, 'amount': -3984}]},
 {'id': 2,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 1879, 'amount': 244},
   {'transaction-id': 3925, 'amount': 203}]})

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 22,
  'name': 'Alice',
  'transactions': [{'transaction-id': 225, 'amount': 1098},
   {'transaction-id': 344, 'amount': 1414},
   {'transaction-id': 811, 'amount': 1125},
   {'transaction-id': 888, 'amount': 1144},
   {'transaction-id': 1166, 'amount': 1200},
   {'transaction-id': 1218, 'amount': 619},
   {'transaction-id': 1293, 'amount': 188},
   {'transaction-id': 1442, 'amount': 1037},
   {'transaction-id': 1512, 'amount': 1155},
   {'transaction-id': 1554, 'amount': 687},
   {'transaction-id': 1605, 'amount': 877},
   {'transaction-id': 1609, 'amount': 938},
   {'transaction-id': 1626, 'amount': 809},
   {'transaction-id': 1915, 'amount': 827},
   {'transaction-id': 2007, 'amount': 967},
   {'transaction-id': 2037, 'amount': 868},
   {'transaction-id': 2119, 'amount': 881},
   {'transaction-id': 2188, 'amount': 695},
   {'transaction-id': 2440, 'amount': 814},
   {'transaction-id': 2469, 'amount': 1110},
   {'transaction-id': 2481, 'amount': 855},
   {'transaction-id': 252

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 78},
 {'name': 'Alice', 'count': 24},
 {'name': 'Alice', 'count': 108},
 {'name': 'Alice', 'count': 61},
 {'name': 'Alice', 'count': 25})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(78, 24, 108, 61, 25)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

58.566

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 225, 'amount': 1098},
  {'transaction-id': 344, 'amount': 1414},
  {'transaction-id': 811, 'amount': 1125},
  {'transaction-id': 888, 'amount': 1144},
  {'transaction-id': 1166, 'amount': 1200},
  {'transaction-id': 1218, 'amount': 619},
  {'transaction-id': 1293, 'amount': 188},
  {'transaction-id': 1442, 'amount': 1037},
  {'transaction-id': 1512, 'amount': 1155},
  {'transaction-id': 1554, 'amount': 687},
  {'transaction-id': 1605, 'amount': 877},
  {'transaction-id': 1609, 'amount': 938},
  {'transaction-id': 1626, 'amount': 809},
  {'transaction-id': 1915, 'amount': 827},
  {'transaction-id': 2007, 'amount': 967},
  {'transaction-id': 2037, 'amount': 868},
  {'transaction-id': 2119, 'amount': 881},
  {'transaction-id': 2188, 'amount': 695},
  {'transaction-id': 2440, 'amount': 814},
  {'transaction-id': 2469, 'amount': 1110},
  {'transaction-id': 2481, 'amount': 855},
  {'transaction-id': 2528, 'amount': 995},
  {'transaction-id': 2815, 'amount': 840},
  {'tra

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 225, 'amount': 1098},
 {'transaction-id': 344, 'amount': 1414},
 {'transaction-id': 811, 'amount': 1125})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1098, 1414, 1125)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

704.2125806782092

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 110), ('Alice', 110), ('Alice', 120), ('Alice', 160), ('Bob', 142), ('Bob', 143), ('Bob', 154), ('Bob', 206), ('Charlie', 148), ('Charlie', 149), ('Charlie', 162), ('Charlie', 216), ('Dan', 127), ('Dan', 129), ('Dan', 139), ('Dan', 186), ('Edith', 78), ('Edith', 80), ('Edith', 85), ('Edith', 113), ('Frank', 121), ('Frank', 121), ('Frank', 132), ('Frank', 176), ('George', 128), ('George', 128), ('George', 140), ('George', 185), ('Hannah', 88), ('Hannah', 88), ('Hannah', 96), ('Hannah', 128), ('Ingrid', 99), ('Ingrid', 99), ('Ingrid', 108), ('Ingrid', 144), ('Jerry', 140), ('Jerry', 141), ('Jerry', 147), ('Jerry', 204), ('Kevin', 109), ('Kevin', 110), ('Kevin', 120), ('Kevin', 159), ('Laura', 66), ('Laura', 66), ('Laura', 72), ('Laura', 96), ('Michael', 55), ('Michael', 55), ('Michael', 60), ('Michael', 80), ('Norbert', 99), ('Norbert', 99), ('Norbert', 108), ('Norbert', 144), ('Oliver', 88), ('Oliver', 88), ('Oliver', 96), ('Oliver', 128), ('Patricia', 66), ('Patricia', 72), 

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 500), ('Bob', 645), ('Charlie', 675), ('Dan', 581), ('Edith', 356), ('Frank', 550), ('George', 581), ('Hannah', 400), ('Ingrid', 450), ('Jerry', 632), ('Kevin', 498), ('Laura', 300), ('Michael', 250), ('Norbert', 450), ('Oliver', 400), ('Patricia', 300), ('Quinn', 400), ('Ray', 412), ('Sarah', 400), ('Tim', 688), ('Ursula', 586), ('Victor', 450), ('Wendy', 440), ('Xavier', 500), ('Yvonne', 350), ('Zelda', 495)]
CPU times: user 131 ms, sys: 7.24 ms, total: 138 ms
Wall time: 464 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Jerry,"[{'transaction-id': 3505, 'amount': 602}, {'tr..."
1,1,Tim,"[{'transaction-id': 830, 'amount': -4332}, {'t..."
2,2,Jerry,"[{'transaction-id': 1879, 'amount': 244}, {'tr..."
3,3,Kevin,"[{'transaction-id': 28, 'amount': 119}, {'tran..."
4,4,Edith,"[{'transaction-id': 20, 'amount': 177}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 154 ms, sys: 10.5 ms, total: 164 ms
Wall time: 1.3 s


name
Alice      500
Bob        645
Charlie    675
Dan        581
Edith      356
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Jerry', 'amount': 602, 'transaction-id': 3505},
 {'id': 0, 'name': 'Jerry', 'amount': 619, 'transaction-id': 7941},
 {'id': 1, 'name': 'Tim', 'amount': -4332, 'transaction-id': 830})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Jerry,602,3505
1,0,Jerry,619,7941
2,1,Tim,-4332,830
3,1,Tim,-3984,9561
4,2,Jerry,244,1879


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 131 ms, sys: 18.9 ms, total: 150 ms
Wall time: 926 ms


name
Alice       29283
Bob         20671
Charlie     26858
Dan         15131
Edith       18925
Frank       26072
George      20297
Hannah      13390
Ingrid      18288
Jerry       30451
Kevin       19094
Laura       12725
Michael      7658
Norbert     18388
Oliver      26245
Patricia     8941
Quinn       11283
Ray         15974
Sarah       30529
Tim         21279
Ursula      21891
Victor      23056
Wendy       31335
Xavier      10664
Yvonne       7379
Zelda       14193
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()